## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [2]:
print (tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/prices.csv", delimiter=",")

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],axis=1,inplace=True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data[:1000]

In [0]:
x = data.iloc[:,0:4]

In [11]:
y = data.iloc[:,4:]/1000000
y

,volume
0,2.1636
1,2.3864
2,2.4895
3,2.0063
4,1.4086
...,...
995,2.1332
996,1.9824
997,37.1528
998,6.5686


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20,random_state=3)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train =np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
Y_train =np.array(Y_train).astype('float32')
Y_test = np.array(Y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(X_train, w, b):
    y_pred = tf.add(tf.matmul(X_train, w), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(Y_train, y_predicted):
    diff = Y_train - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(X_train, Y_train, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(X_train, w, b)
        current_loss = loss(Y_train, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [23]:
for i in range(100):    
    w, b = train(X_train, Y_train, w, b)
    print('Current Training Loss on iteration', i, ' -- ', loss(Y_train, prediction(X_train, w, b)).numpy())

Current Training Loss on iteration 0  --  197.80934
Current Training Loss on iteration 1  --  195.78816
Current Training Loss on iteration 2  --  193.92549
Current Training Loss on iteration 3  --  192.2088
Current Training Loss on iteration 4  --  190.62675
Current Training Loss on iteration 5  --  189.1688
Current Training Loss on iteration 6  --  187.82506
Current Training Loss on iteration 7  --  186.58664
Current Training Loss on iteration 8  --  185.44536
Current Training Loss on iteration 9  --  184.39355
Current Training Loss on iteration 10  --  183.42416
Current Training Loss on iteration 11  --  182.5308
Current Training Loss on iteration 12  --  181.70747
Current Training Loss on iteration 13  --  180.94868
Current Training Loss on iteration 14  --  180.24937
Current Training Loss on iteration 15  --  179.60492
Current Training Loss on iteration 16  --  179.01096
Current Training Loss on iteration 17  --  178.4636
Current Training Loss on iteration 18  --  177.9591
Current 

### Get the shapes and values of W and b

In [31]:
print ('shape of W : ', w)

shape of W :  tf.Tensor(
[[1.281082 ]
 [1.3182442]
 [1.2747002]
 [1.325066 ]], shape=(4, 1), dtype=float32)


In [33]:
print ('shape of B : ', b)

shape of B :  tf.Tensor([2.600025], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [34]:
pred=prediction(X_test,w,b)
pred

<tf.Tensor: id=930, shape=(200, 1), dtype=float32, numpy=
array([[5.1999063],
       [5.1996746],
       [5.199752 ],
       [5.1996684],
       [5.1997194],
       [5.1998873],
       [5.199869 ],
       [5.1998224],
       [5.1996727],
       [5.1996584],
       [5.1998076],
       [5.199914 ],
       [5.199604 ],
       [5.1996017],
       [5.1999097],
       [5.1998696],
       [5.1999326],
       [5.199791 ],
       [5.1999025],
       [5.199731 ],
       [5.1998134],
       [5.1996107],
       [5.1995883],
       [5.199662 ],
       [5.199732 ],
       [5.199565 ],
       [5.1998777],
       [5.199174 ],
       [5.1993084],
       [5.1996126],
       [5.199934 ],
       [5.1997857],
       [5.199943 ],
       [5.1998982],
       [5.199571 ],
       [5.1998544],
       [5.1996512],
       [5.199623 ],
       [5.199887 ],
       [5.1996355],
       [5.1999006],
       [5.199011 ],
       [5.199943 ],
       [5.1998158],
       [5.1996307],
       [5.1996164],
       [5.199826 ],
  

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("/content/drive/My Drive/Colab Notebooks/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
iris.Species = le.fit_transform(iris.Species)

In [0]:
y_iris = tf.keras.utils.to_categorical(iris.Species, num_classes=3)

### Splitting the data into feature set and target set

In [47]:
X_iris = iris.iloc[:,1:5]
#y_iris = iris.Species
y_iris.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:

# Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

# Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X_iris, y_iris, test_size=0.20,random_state=3)

### Model Training 

In [52]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=30)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 122us/sample - loss: 0.4367 - accuracy: 0.8917 - val_loss: 0.4089 - val_accuracy: 0.9667
Epoch 2/30
120/120 [==============================] - 0s 122us/sample - loss: 0.4161 - accuracy: 0.9500 - val_loss: 0.4010 - val_accuracy: 0.9667
Epoch 3/30
120/120 [==============================] - 0s 150us/sample - loss: 0.4084 - accuracy: 0.9167 - val_loss: 0.3937 - val_accuracy: 0.9667
Epoch 4/30
120/120 [==============================] - 0s 161us/sample - loss: 0.4005 - accuracy: 0.9500 - val_loss: 0.3874 - val_accuracy: 0.9333
Epoch 5/30
120/120 [==============================] - 0s 123us/sample - loss: 0.4003 - accuracy: 0.9250 - val_loss: 0.3806 - val_accuracy: 0.9333
Epoch 6/30
120/120 [==============================] - 0s 122us/sample - loss: 0.3886 - accuracy: 0.9333 - val_loss: 0.3755 - val_accuracy: 0.9333
Epoch 7/30
120/120 [==============================] - 0s 102us/sample - loss: 0

### Model Prediction

In [53]:
model.predict(testX)

array([[9.47874308e-01, 5.18831573e-02, 2.42517883e-04],
       [9.31803524e-01, 6.77853301e-02, 4.11158981e-04],
       [9.55880642e-01, 4.39987704e-02, 1.20557917e-04],
       [9.24635768e-01, 7.49417245e-02, 4.22528246e-04],
       [9.82382894e-01, 1.76030789e-02, 1.40669445e-05],
       [7.95998585e-06, 3.96690145e-02, 9.60323036e-01],
       [2.15298664e-02, 5.63885689e-01, 4.14584428e-01],
       [9.52743351e-01, 4.70608734e-02, 1.95749904e-04],
       [4.57295624e-04, 1.74907431e-01, 8.24635267e-01],
       [3.82506102e-02, 7.44365990e-01, 2.17383415e-01],
       [3.60853299e-02, 6.58095598e-01, 3.05819154e-01],
       [9.42378521e-01, 5.72864041e-02, 3.35081771e-04],
       [3.49377319e-02, 6.57356560e-01, 3.07705641e-01],
       [1.61005799e-02, 5.15447080e-01, 4.68452424e-01],
       [3.62272956e-04, 1.71186119e-01, 8.28451633e-01],
       [9.68532145e-01, 3.13946083e-02, 7.32468761e-05],
       [2.99634179e-03, 3.51730257e-01, 6.45273447e-01],
       [2.33351035e-04, 1.78504

### Save the Model

In [0]:
import pickle
filename = '/content/drive/My Drive/Colab Notebooks/finalized_model.h5'

# new_model = tf.keras.models.load_model('/content/finalized_model.h5')
# new_model.summary()

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?